<a href="https://colab.research.google.com/github/GabBogni/Angular_EAD1/blob/master/Programa_de_Representacao_Grafica_dos_Indicadores_de_Fluxo_de_Alunos_dos_Cursos_da_EESC_e_ICMC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

O código abaixo precisa ser executado para conseguir acesso as pastas do drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


O código abaixo precisa ser executado para a maioria das funções funcionar, pois possui recursos que são usados por elas.

Existe uma lista de cursos considerados que são usadas em diversas funções.

Os cursos considerados contém a palavra ‘Engenharia’.

Alguns cursos não devem ser considerados como:
<ul>
  <li>Licenciatura em Ciências Exatas/Habilitação em Matemática </li>
  <li>Licenciatura em Ciências Exatas/Licenciatura em Ciências Exatas - Núcleo Geral</li>
  <li>Licenciatura em Ciências Exatas/Habilitação em Física</li>
  <li>Engenharia Civil/Dupla Formação EESC - IAU</li>
  <li>Engenharia Mecânica/Engenharia Mecânica - Núcleo Geral</li>
  <li>Engenharia Mecânica/Ênfase Mecânica Plena</li>
  <li>Licenciatura em Ciências Exatas/Habilitação em Química</li>
</ul>


In [3]:
import csv
import matplotlib.pyplot as plt
import pandas
import numpy
from enum import Enum
from datetime import datetime, date

class Colunas_Tabela_EESC_ICMC(Enum):
  ANO_DO_INGRESSO = 0
  CODIGO_DO_CURSO = 1
  NOME_DO_CURSO = 2
  CODIGO_DA_HABILITACAO = 3
  NOME_DA_HABILITACAO = 4
  DATA_DO_ENCERRAMENTO = 12
  TIPO_DO_ENCERRAMENTO = 13

NUMERO_DA_PRIMEIRA_LINHA_COM_DADOS = 1

valores_da_coluna_ano_do_ingresso = []
valores_da_coluna_codigo_do_curso = []
valores_da_coluna_nome_do_curso = []
valores_da_coluna_nome_da_habilitacao = []
valores_da_coluna_codigo_da_habilitacao = []
valores_da_coluna_tipo_do_encerramento = []
valores_da_coluna_data_do_encerramento = []


#-------------------------------------------- Abrindo arquivo csv e coletando informações --------------------------------------------

diretorio_e_nome_do_arquivo = '/content/drive/MyDrive/Estagio USP/Tabelas Bases/Cópia de (Gerada 02.06.2022) [EESC _ ICMC] Ingressantes SISU_FUVEST - Ingressantes de 2015-2022.xlsx'

with open(diretorio_e_nome_do_arquivo, 'r+', encoding = 'UTF-8', newline = '') as arquivo: 
  objeto_reader = csv.reader(arquivo, delimiter = ',')# Lendo o arquivo csv com delimitador igual a ','

  for numero_da_linha in range(NUMERO_DA_PRIMEIRA_LINHA_COM_DADOS, len(objeto_reader)):
    valores_da_coluna_ano_do_ingresso.append(objeto_reader[numero_da_linha][Colunas_Tabela_EESC_ICMC.ANO_DO_INGRESSO.value])
    valores_da_coluna_codigo_do_curso.append(objeto_reader[numero_da_linha][Colunas_Tabela_EESC_ICMC.CODIGO_DO_CURSO.value])
    valores_da_coluna_codigo_da_habilitacao.append(objeto_reader[numero_da_linha][Colunas_Tabela_EESC_ICMC.CODIGO_DA_HABILITACAO.value])
    valores_da_coluna_nome_do_curso.append(objeto_reader[numero_da_linha][Colunas_Tabela_EESC_ICMC.NOME_DO_CURSO.value])
    valores_da_coluna_nome_da_habilitacao.append(objeto_reader[numero_da_linha][Colunas_Tabela_EESC_ICMC.NOME_DA_HABILITACAOvalue])
    valores_da_coluna_tipo_do_encerramento.append(objeto_reader[numero_da_linha][Colunas_Tabela_EESC_ICMC.TIPO_DO_ENCERRAMENTO.value])
    valores_da_coluna_data_do_encerramento.append(objeto_reader[numero_da_linha][Colunas_Tabela_EESC_ICMC.DATA_DO_ENCERRAMENTO.value])

#-------------------------------------------- Funções necessárias por uso dentro de outras funções --------------------------------------------

def verificar_igualdade_entre_codigos(curso_ou_unidade, **parametros_palavra_chave):
    if curso_ou_unidade == 'curso':
      dicionario_TESTE_de_codigoS_de_UM_curso_e_UMA_habilitacao = parametros_palavra_chave['dicionario_TESTE_de_codigoS_de_UM_curso_e_UMA_habilitacao']
      dicionario_MODELO_de_codigoS_de_UM_curso_e_UMA_habilitacao = parametros_palavra_chave['dicionario_MODELO_de_codigoS_de_UM_curso_e_UMA_habilitacao']
      if(
        (dicionario_TESTE_de_codigoS_de_UM_curso_e_UMA_habilitacao['codigo_do_curso'] == dicionario_MODELO_de_codigoS_de_UM_curso_e_UMA_habilitacao['codigo_do_curso'] and 
        dicionario_MODELO_de_codigoS_de_UM_curso_e_UMA_habilitacao.get('codigo_da_habilitacao')  == None)
        or 
        (dicionario_TESTE_de_codigoS_de_UM_curso_e_UMA_habilitacao['codigo_do_curso'] == dicionario_MODELO_de_codigoS_de_UM_curso_e_UMA_habilitacao['codigo_do_curso'] and
        dicionario_TESTE_de_codigoS_de_UM_curso_e_UMA_habilitacao['codigo_da_habilitacao'] == dicionario_MODELO_de_codigoS_de_UM_curso_e_UMA_habilitacao['codigo_da_habilitacao'])
        ):
        return True
      return False
    else:
      if parametros_palavra_chave['codigo_TESTE_da_unidade'] == parametros_palavra_chave['codigo_MODELO_da_unidade']:
        return True
      return False

def retornar_codigos_de_cursos_e_habilitacoes_considerados_da_EESC(codigoS_de_cursoS_e_habilitacOES_NAO_consideradoS):
  
  codigoS_de_cursoS_e_habilitacOES_consideradoS = []
  codigoS_de_cursos_NAO_consideradoS = []

  for codigo_do_curso, codigo_da_habilitacao, nome_do_curso, nome_da_habilitacao in zip(valores_da_coluna_codigo_do_curso, 
                                                                                        valores_da_coluna_codigo_da_habilitacao,
                                                                                        valores_da_coluna_nome_do_curso, 
                                                                                        valores_da_coluna_nome_da_habilitacao):
    
      codigoS_de_UM_curso_e_UMA_habilitacao_de_um_ALUNO = {'codigo_do_curso': codigo_do_curso, 'codigo_da_habilitacao':codigo_da_habilitacao,
                        'nome_do_curso': nome_do_curso, 'nome_da_habilitacao': nome_da_habilitacao}

      validade_do_curso_teste = True

      for codigoS_de_UM_curso_e_UMA_habilitacao_NAO_consideradoS in codigoS_de_cursoS_e_habilitacOES_NAO_consideradoS:

        if verificar_igualdade_entre_codigos(codigoS_de_UM_curso_e_UMA_habilitacao_de_um_ALUNO, codigoS_de_UM_curso_e_UMA_habilitacao_NAO_consideradoS) == True:
          validade_do_curso_teste = False
          break

      if validade_do_curso_teste == True and codigoS_de_cursoS_e_habilitacOES_consideradoS.count(codigoS_de_UM_curso_e_UMA_habilitacao_de_um_ALUNO) == 0:
        codigoS_de_cursoS_e_habilitacOES_consideradoS.append(codigoS_de_UM_curso_e_UMA_habilitacao_de_um_ALUNO)

  return codigoS_de_cursoS_e_habilitacOES_consideradoS

def tratar_e_retornar_dados_de_alunos_ingressados_em_T_para_um_CURSO(ano_de_ingresso_dos_alunos, codigoS_de_um_curso_e_habilitacao, data_para_correcao_da_data_de_conclusao):
  T = ano_de_ingresso_dos_alunos

  j = codigoS_de_um_curso_e_habilitacao

  alunos_ingressados_em_T = []

  for codigo_do_curso, codigo_da_habilitacao, ano_do_ingresso, data_do_encerramento, tipo_do_encerramento in zip(valores_da_coluna_codigo_do_curso,
                                                                                                                 valores_da_coluna_ano_do_ingresso, 
                                                                                                                 valores_da_coluna_data_do_encerramento,
                                                                                                                 valores_da_coluna_tipo_do_encerramento):
    
    codigoS_de_UM_curso_e_habilitacao_de_UM_aluno = {'codigo_curso':codigo_do_curso, 'codigo_habilitacao':codigo_da_habilitacao}

    if verificar_igualdade_entre_codigos(codigoS_de_UM_curso_e_habilitacao_de_UM_aluno, codigoS_de_um_curso_e_habilitacao) == True and int(ano_do_ingresso) == T:

      if data_de_encerramento != '':
        data_de_encerramento =  datetime.strptime(data_de_encerramento,'%d/%m/%Y').date()

        if(tipo_do_encerramento.find('Conclusão')!= -1 and 
            data_para_correcao_da_data_de_conclusao != None and 
            data_de_encerramento <= date(data_de_encerramento.year, data_para_correcao_da_data_de_conclusao.month, data_para_correcao_da_data_de_conclusao.day)
        ):
          data_de_encerramento = date(data_de_encerramento.year - 1, 12, 31)

      alunos_ingressados_em_T.append({'ano_do_ingresso':ano_do_ingresso,
                                      'data_de_encerramento':data_de_encerramento, 
                                      'tipo_do_encerramento':tipo_do_encerramento
                                      })
 
  return alunos_ingressados_em_T

"""
def tratar_e_retornar_dados_de_alunos_ingressados_em_T_para_uma_UNIDADE(ano_de_ingresso_dos_alunos, codigo_da_unidade, data_para_correcao_da_data_de_conclusao):
  T = ano_de_ingresso_dos_alunos
  
  nomes_dos_cursos_da_unidade = codigos_de_unidades_e_nomes_de_seus_cursos[codigo_da_unidade] # codigos_de_unidades_e_nomes_de_seus_cursos é uma variável global

  coluna_cursos = spreadsheet.col_values(Colunas_Tabela_EESC_ICMC.CURSO.value)[1:]
  coluna_anos_de_ingresso = spreadsheet.col_values(Colunas_Tabela_EESC_ICMC.ANO.value)[1:]
  coluna_data_de_encerramento = spreadsheet.col_values(Colunas_Tabela_EESC_ICMC.ENCERRAMENTO.value)[1:]
  coluna_tipo_de_encerramento = spreadsheet.col_values(Colunas_Tabela_EESC_ICMC.TIPO_ENCERRAMENTO.value)[1:]

  alunos_ingressados_em_T = []

  for curso, ano_de_ingresso, data_de_encerramento, tipo_de_encerramento in zip(coluna_cursos, coluna_anos_de_ingresso, coluna_data_de_encerramento, coluna_tipo_de_encerramento):
    
    if curso in nomes_dos_cursos_da_unidade and int(ano_de_ingresso) == T:
    
      if data_de_encerramento != '':

        if type(data_de_encerramento) == float or type(data_de_encerramento) == int:
          data_de_encerramento =  xlrd.xldate.xldate_as_datetime(data_de_encerramento, workbook.datemode).date()
        else:
          data_de_encerramento =  datetime.strptime(data_de_encerramento,'%d/%m/%Y').date()

        if(tipo_de_encerramento.find('Conclusão')!= -1 and 
            data_para_correcao_da_data_de_conclusao != None and 
            data_de_encerramento <= date(data_de_encerramento.year, data_para_correcao_da_data_de_conclusao.month, data_para_correcao_da_data_de_conclusao.day)
        ):
          data_de_encerramento = date(data_de_encerramento.year - 1, 12, 31)

      alunos_ingressados_em_T.append((ano_de_ingresso, data_de_encerramento, tipo_de_encerramento))
 
  return alunos_ingressados_em_T
"""

#-------------------------------------------- Função para validar dados, usado em outras funções --------------------------------------------

def gerar_tabela_para_validar_dados(dicionario_contendo_cabecalho_coluna_e_valores, titulo_da_tabela, transpor_dados_da_tabela):
  dataframe = pandas.DataFrame(data=dicionario_contendo_cabecalho_coluna_e_valores)
  if(transpor_dados_da_tabela):
    dataframe = dataframe.transpose()
  print('\n{}\n'.format(titulo_da_tabela))
  display(dataframe)
  print('')

#-------------------------------------------- Função para facilitar o ato de salvar uma Figura --------------------------------------------

def salvar_Figura(string_do_diretorio_da_pasta, string_nome_do_arquivo, string_da_extensao_do_arquivo, resolucao_da_figura):
  string_nome_do_arquivo = string_nome_do_arquivo.replace('/','|')
  diretorio = string_do_diretorio_da_pasta+string_nome_do_arquivo+'.'+string_da_extensao_do_arquivo
  plt.savefig(diretorio, dpi = resolucao_da_figura, bbox_inches = 'tight')

#-------------------------------------------- Variaveis importantes para o funções  --------------------------------------------
cursos_nao_considerados = [
    {'codigo_curso':90011},
    {'codigo_curso':18062},
    {'codigo_curso':18023, 'codigo_habilitacao':100},
    {'codigo_curso':18023, 'codigo_habilitacao':200},
]

codigos_de_cursos_e_habilitacoes_considerados_da_EESC = retornar_codigos_de_cursos_e_habilitacoes_considerados_da_EESC(cursos_nao_considerados)
anos_de_ingresso_considerados = [2015,2016,2017,2018,2019,2020,2021,2022]

plt.rcParams['font.size'] = 15


TypeError: ignored

###calcular_taxas_de_permanencia_de_varios_anos_para_um_CURSO(anos_de_referencia, alunos_ingressados_em_T, data_da_geracao_da_tabela)
---
O objetivo da função é calcular a taxa de permanência de cada ano ao longo dos anos para um CURSO.

In [ ]:
def calcular_taxas_de_permanencia_de_varios_anos_para_um_CURSO(anos_de_referencia, alunos_ingressados_em_T, data_da_geracao_da_tabela):

  def calcular_taxa_de_permanencia_de_um_ano_para_um_CURSO(t, alunos_ingressados_em_T):        
    somatoria_IG_do_ano_T = len(alunos_ingressados_em_T)
    somatoria_de_alunos_com_vinculo_ativo_no_ano_t = 0
    somatoria_de_Cur_do_ano_t = None # Não é possivel calcular através do banco de dados fornecido
    somatoria_de_mTr_do_ano_t = None # Não é possivel calcular através do banco de dados fornecido
    somatoria_de_Fal_do_ano_T_ate_ano_t = 0

    for ano_de_ingresso, data_de_encerramento, tipo_de_encerramento in alunos_ingressados_em_T:

      if data_de_encerramento == '':
        somatoria_de_alunos_com_vinculo_ativo_no_ano_t +=1
      else:
        if t < data_de_encerramento.year:
          somatoria_de_alunos_com_vinculo_ativo_no_ano_t +=1
        
        elif tipo_de_encerramento.find('Falecimento')!= -1:
          somatoria_de_Fal_do_ano_T_ate_ano_t+=1

    taxa_de_permanencia_do_ano_t = (somatoria_de_alunos_com_vinculo_ativo_no_ano_t / (somatoria_IG_do_ano_T - somatoria_de_Fal_do_ano_T_ate_ano_t)) * 100

    dados_calculados_do_ano_t = {
    't' : t,
    'taxa_de_permanencia_do_ano_t' : taxa_de_permanencia_do_ano_t,
    'somatoria_de_alunos_com_vinculo_ativo_no_ano_t' : somatoria_de_alunos_com_vinculo_ativo_no_ano_t,
    'somatoria_de_Fal_do_ano_T_ate_ano_t' : somatoria_de_Fal_do_ano_T_ate_ano_t,
    'somatoria_IG_do_ano_T' : somatoria_IG_do_ano_T,
    }

    return dados_calculados_do_ano_t

  #-------------------------Conteúdo da Função Principal ------------------------------
  tS = anos_de_referencia 

  if (data_da_geracao_da_tabela.day < 31 or data_da_geracao_da_tabela.month < 12) and max(tS) >= data_da_geracao_da_tabela.year:
    return {'estado':False, 'resposta':'Erro, no caso de uma data diferente de 31/12/ano, todo ano de referência deve ser menor que o ano da data de geração da tabela.'}
  
  elif (data_da_geracao_da_tabela.day == 31 and data_da_geracao_da_tabela.month == 12) and max(tS) > data_da_geracao_da_tabela.year:
    return {'estado':False, 'resposta':'Erro, no caso de uma data igual a 31/12/ano, todo ano de referência deve ser menor ou igual ao ano da data de geração da tabela.'}

  dados_calculados_de_anos = []

  taxas_de_permanencia_de_anos = []

  for t in tS:

    dados_calculados_do_ano_t = calcular_taxa_de_permanencia_de_um_ano_para_um_CURSO(t, alunos_ingressados_em_T)

    dados_calculados_de_anos.append(dados_calculados_do_ano_t)

    taxas_de_permanencia_de_anos.append(dados_calculados_do_ano_t['taxa_de_permanencia_do_ano_t'])

  dados_calculados_de_anos_por_categoria = {
  't' : [],
  'Taxa de Permanência do Ano t (%)' : [] ,
  'Somatória de Alunos com Vínculos Ativos do Ano t' : [] ,
  'Somatória de Alunos Falecidos do Ano T até Ano t' : [],
  'Somatória de Alunos Ingressados em T': []
  }

  for dados_calculados_do_ano_t in dados_calculados_de_anos:

    dados_calculados_de_anos_por_categoria ['t'].append(dados_calculados_do_ano_t['t'])
    dados_calculados_de_anos_por_categoria ['Taxa de Permanência do Ano t (%)'].append(dados_calculados_do_ano_t['taxa_de_permanencia_do_ano_t'])
    dados_calculados_de_anos_por_categoria ['Somatória de Alunos com Vínculos Ativos do Ano t'].append(dados_calculados_do_ano_t['somatoria_de_alunos_com_vinculo_ativo_no_ano_t'])
    dados_calculados_de_anos_por_categoria ['Somatória de Alunos Falecidos do Ano T até Ano t'].append(dados_calculados_do_ano_t['somatoria_de_Fal_do_ano_T_ate_ano_t'])
    dados_calculados_de_anos_por_categoria ['Somatória de Alunos Ingressados em T'].append(dados_calculados_do_ano_t['somatoria_IG_do_ano_T'])

  gerar_tabela_para_validar_dados(dados_calculados_de_anos_por_categoria, 'Taxa de Permanência dos Alunos', False)

  return {'estado': True, 'resposta': taxas_de_permanencia_de_anos}

###calcular_taxas_de_permanencia_de_varios_anos_para_um_CURSO_VERSAO_2(anos_de_referencia, alunos_ingressados_em_T, data_da_geracao_da_tabela)
---
O objetivo da função é calcular a taxa de permanência de cada ano ao longo dos anos para um CURSO.


In [ ]:
def calcular_taxas_de_permanencia_de_varios_anos_para_um_CURSO_VERSAO_2(anos_de_referencia, alunos_ingressados_em_T, data_da_geracao_da_tabela):

  def calcular_taxa_de_permanencia_de_um_ano_para_um_CURSO_VERSAO_2(t, alunos_ingressados_em_T, somatoria_de_alunos_com_vinculo_ativo_do_ano_t_menos_1):
    
    somatoria_IG_do_ano_T = len(alunos_ingressados_em_T) 
    somatoria_de_Cur_do_ano_t = None # Não é possivel calcular através do banco de dados fornecido
    somatoria_de_mTr_do_ano_t = None # Não é possivel calcular através do banco de dados fornecido
    somatoria_de_alunos_tranferidos_do_ano_t = 0
    somatoria_de_alunos_desistentes_do_ano_t = 0
    somatoria_de_alunos_formados_do_ano_t = 0
    somatoria_de_Fal_do_ano_t = 0
    somatoria_de_Fal_do_ano_T_ate_t = 0

    for ano_de_ingresso, data_de_encerramento, tipo_de_encerramento in alunos_ingressados_em_T:

      if data_de_encerramento != '':

        if t == data_de_encerramento.year:

          if tipo_de_encerramento.find('Transferência')!= -1:
            somatoria_de_alunos_tranferidos_do_ano_t +=1

          elif (tipo_de_encerramento.find('Abandono 2 semestres sem matrícula')!= -1 or
                tipo_de_encerramento.find('Cancelamento 0 crédito')!= -1 or
                tipo_de_encerramento.find('Cancelamento 0 crédito em dois semestres')!= -1 or
                tipo_de_encerramento.find('Cancelamento menos 20% créd 2 semestres')!= -1 or
                tipo_de_encerramento.find('Cancelamento menos 20% dos créditos')!= -1 or
                tipo_de_encerramento.find('Cancelamento trancamento 4 semestres')!= -1 or
                tipo_de_encerramento.find('Desistência a pedido')!= -1 or
                tipo_de_encerramento.find('Ingressante sem Frequência')!= -1 or
                tipo_de_encerramento.find('Encerramento novo ingresso')!= -1 or
                tipo_de_encerramento.find('Cancelamento outras IES (Lei Federal)')!= -1 or
                tipo_de_encerramento.find('Não cumpriu normas Decreto (PEC-G)')!= -1 or
                tipo_de_encerramento.find('Opção Currículo')!= -1 or
                tipo_de_encerramento.find('Liminar cassada')!= -1):
            
            somatoria_de_alunos_desistentes_do_ano_t +=1
          
          elif (tipo_de_encerramento.find('Conclusão')!= -1 or
                tipo_de_encerramento.find('Término')!= -1):
            
            somatoria_de_alunos_formados_do_ano_t +=1
          
          elif tipo_de_encerramento.find('Falecimento') != -1:
            somatoria_de_Fal_do_ano_t +=1
          
        if t >= data_de_encerramento.year and tipo_de_encerramento.find('Falecimento') != -1:
          somatoria_de_Fal_do_ano_T_ate_t +=1

    somatoria_de_alunos_com_vinculo_ativo_no_ano_t = (somatoria_de_alunos_com_vinculo_ativo_do_ano_t_menos_1
                                                                  - somatoria_de_alunos_tranferidos_do_ano_t
                                                                  - somatoria_de_alunos_desistentes_do_ano_t
                                                                  - somatoria_de_alunos_formados_do_ano_t
                                                                  - somatoria_de_Fal_do_ano_t)
    
    taxa_de_permanencia_do_ano_t = ( somatoria_de_alunos_com_vinculo_ativo_no_ano_t / (somatoria_IG_do_ano_T - somatoria_de_Fal_do_ano_T_ate_t)) * 100 

    dados_calculados_do_ano_t = {
      't' : t,
      'taxa_de_permanencia_do_ano_t' : taxa_de_permanencia_do_ano_t,
      'somatoria_de_alunos_com_vinculo_ativo_no_ano_t' : somatoria_de_alunos_com_vinculo_ativo_no_ano_t,
      'somatoria_de_alunos_tranferidos_do_ano_t' : somatoria_de_alunos_tranferidos_do_ano_t,
      'somatoria_de_alunos_desistentes_do_ano_t' : somatoria_de_alunos_desistentes_do_ano_t,
      'somatoria_de_alunos_formados_do_ano_t' : somatoria_de_alunos_formados_do_ano_t ,
      'somatoria_de_Fal_do_ano_t' : somatoria_de_Fal_do_ano_t,
      'somatoria_de_Fal_do_ano_T_ate_t' : somatoria_de_Fal_do_ano_T_ate_t,
      'somatoria_IG_do_ano_T' : somatoria_IG_do_ano_T
    }

    return dados_calculados_do_ano_t


  #-------------------------Conteúdo da Função Principal ------------------------------

  tS = anos_de_referencia 

  if (data_da_geracao_da_tabela.day < 31 or data_da_geracao_da_tabela.month < 12) and max(tS) >= data_da_geracao_da_tabela.year:
    return {'estado':False, 'resposta':'Erro, no caso de uma data diferente de 31/12/ano, todo ano de referência deve ser menor que o ano da data de geração da tabela.'}
  
  elif (data_da_geracao_da_tabela.day == 31 and data_da_geracao_da_tabela.month == 12) and max(tS) > data_da_geracao_da_tabela.year:
    return {'estado':False, 'resposta':'Erro, no caso de uma data igual a 31/12/ano, todo ano de referência deve ser menor ou igual ao ano da data de geração da tabela.'}
    
  dados_calculados_de_anos = []

  taxas_de_permanencia_de_anos = []

  somatoria_de_alunos_com_vinculo_ativo_do_ano_t_menos_1 = len(alunos_ingressados_em_T) # Essa atribuição é feita pois no ano T, não existe t-1

  for t in tS:

    dados_calculados_do_ano_t = calcular_taxa_de_permanencia_de_um_ano_para_um_CURSO_VERSAO_2(t, alunos_ingressados_em_T, somatoria_de_alunos_com_vinculo_ativo_do_ano_t_menos_1)

    dados_calculados_de_anos.append(dados_calculados_do_ano_t)

    taxas_de_permanencia_de_anos.append(dados_calculados_do_ano_t['taxa_de_permanencia_do_ano_t'])

    somatoria_de_alunos_com_vinculo_ativo_do_ano_t_menos_1 = dados_calculados_do_ano_t['somatoria_de_alunos_com_vinculo_ativo_no_ano_t']

    dados_calculados_de_anos_por_categoria = {
      't' : [ ],
      'Taxa de Permanência do Ano t (%)' : [] ,
      'Somatória de Alunos com Vínculo Ativo do Ano t' : [] ,
      'Somatória de Alunos Transferidos do Ano t' : [] ,
      'Somatória de Alunos Desistentes do Ano t' : [] ,
      'Somatória de Alunos Formados do Ano t' : [] ,
      'Somatória de Alunos Falecidos do Ano t' : [],
      'Somatória de Alunos Falecidos do Ano T até Ano t' : [],
      'Somatória de Alunos Ingressados em T': []
    }

  for dados_calculados_do_ano_t in dados_calculados_de_anos:

    dados_calculados_de_anos_por_categoria ['t'].append(dados_calculados_do_ano_t['t'])
    dados_calculados_de_anos_por_categoria ['Taxa de Permanência do Ano t (%)'].append(dados_calculados_do_ano_t['taxa_de_permanencia_do_ano_t'])
    dados_calculados_de_anos_por_categoria ['Somatória de Alunos com Vínculo Ativo do Ano t'].append(dados_calculados_do_ano_t['somatoria_de_alunos_com_vinculo_ativo_no_ano_t'])
    dados_calculados_de_anos_por_categoria ['Somatória de Alunos Transferidos do Ano t'].append(dados_calculados_do_ano_t['somatoria_de_alunos_tranferidos_do_ano_t'])
    dados_calculados_de_anos_por_categoria ['Somatória de Alunos Desistentes do Ano t'].append(dados_calculados_do_ano_t['somatoria_de_alunos_desistentes_do_ano_t'])
    dados_calculados_de_anos_por_categoria ['Somatória de Alunos Formados do Ano t'].append(dados_calculados_do_ano_t['somatoria_de_alunos_formados_do_ano_t'])
    dados_calculados_de_anos_por_categoria ['Somatória de Alunos Falecidos do Ano t'].append(dados_calculados_do_ano_t['somatoria_de_Fal_do_ano_t'])
    dados_calculados_de_anos_por_categoria ['Somatória de Alunos Falecidos do Ano T até Ano t'].append(dados_calculados_do_ano_t['somatoria_de_Fal_do_ano_T_ate_t'])
    dados_calculados_de_anos_por_categoria ['Somatória de Alunos Ingressados em T'].append(dados_calculados_do_ano_t['somatoria_IG_do_ano_T'])

  gerar_tabela_para_validar_dados(dados_calculados_de_anos_por_categoria, 'Taxa de Permanência dos Alunos', False)

  return {'estado':True, 'resposta':taxas_de_permanencia_de_anos}
  

###calcular_e_plotar_grafico_de_taxas_de_permanencia_de_varios_anos_para_um_CURSO(ano_de_ingresso_dos_alunos, nome_do_curso, anos_de_referencia, data_da_geracao_da_tabela)
---
O objetivo da função é criar um gráfico para um CURSO contendo a taxa de permanência para cada ano ao longo dos anos. Para isso utiliza a função definida no código anterior para realizar os cálculos.

In [ ]:
def calcular_e_plotar_grafico_de_taxas_de_permanencia_de_varios_anos_para_um_CURSO(ano_de_ingresso_dos_alunos, nome_do_curso, anos_de_referencia, data_da_geracao_da_tabela, data_para_correcao_da_data_de_conclusao):
  
  alunos_ingressados_em_T = tratar_e_retornar_dados_de_alunos_ingressados_em_T_para_um_CURSO(ano_de_ingresso_dos_alunos, nome_do_curso, data_para_correcao_da_data_de_conclusao)
  
  resposta_da_funcao = calcular_taxas_de_permanencia_de_varios_anos_para_um_CURSO_VERSAO_2(anos_de_referencia, alunos_ingressados_em_T, data_da_geracao_da_tabela)
  
  if resposta_da_funcao['estado'] == False:
    print(resposta_da_funcao['resposta'])
    return

  taxas_de_permanencia_de_anos = resposta_da_funcao['resposta']

  figure = plt.figure(figsize = (9,6))
  axe = figure.add_axes([0,0,1,1])
  axe.plot(anos_de_referencia, taxas_de_permanencia_de_anos, 'D-b')
  axe.set_xlabel('Anos')
  axe.set_ylabel('Taxas de Permanência de Anos')
  axe.set_title('Gráfico de Taxas de Permanência de Anos do Curso de '+nome_do_curso)

  incremento_x = (axe.get_xlim()[1] - axe.get_xlim()[0]) * 0.01 # para facilitar visualização dos valores plotados em relação ao eixo x, proporcional ao tamanho do eixo X
  incremento_y = (axe.get_ylim()[1] - axe.get_ylim()[0]) * 0.01 # para facilitar visualização dos valores plotados em relação ao eixo y, proporcional ao tamanho do eixo Y

  # Utilizando função text do módulo pyplot para inserir textos em coordenadas no gráfico.
  for coordenada_x, coordenada_y in zip(anos_de_referencia, taxas_de_permanencia_de_anos):
    plt.text(coordenada_x+incremento_x, coordenada_y+incremento_y, '{:.2f}%'.format(coordenada_y))

  plt.show()

In [ ]:
#Testes
#print(nomes_dos_cursos_considerados_da_EESC)
for curso in nomes_dos_cursos_considerados_da_EESC:
  calcular_e_plotar_grafico_de_taxas_de_permanencia_de_varios_anos_para_um_CURSO(2015, curso, [2015,2016,2017,2018,2019,2020], 
                                                                                 data_da_geracao_da_tabela = date(2021,4,21), 
                                                                                 data_para_correcao_da_data_de_conclusao = date(9999,5,31))


###calcular_taxas_de_conclusao_acumulada_de_varios_anos_para_um_CURSO(anos_de_referencia, alunos_ingressados_em_T, data_da_geracao_da_tabela)
---
O objetivo da função é calcular a taxa de conclusão acumulada de cada ano ao longo dos anos para um CURSO.

In [ ]:
def calcular_taxas_de_conclusao_acumulada_de_varios_anos_para_um_CURSO(anos_de_referencia, alunos_ingressados_em_T, data_da_geracao_da_tabela):

  def calcular_taxa_de_conclusao_acumulada_de_um_ano_para_um_CURSO(t, alunos_ingressados_em_T):
    
    somatoria_IG_do_ano_T = len(alunos_ingressados_em_T)
    somatoria_de_alunos_formados_do_ano_T_ate_ano_t = 0
    somatoria_de_Fal_do_ano_T_ate_ano_t = 0 

    for ano_de_ingresso, data_de_encerramento, tipo_de_encerramento in alunos_ingressados_em_T:

      if data_de_encerramento != '':

        if t >= data_de_encerramento.year:

          if (tipo_de_encerramento.find('Conclusão')!= -1 or
                tipo_de_encerramento.find('Término')!= -1):
                  somatoria_de_alunos_formados_do_ano_T_ate_ano_t +=1
          
          elif tipo_de_encerramento.find('Falecimento') != -1:
            somatoria_de_Fal_do_ano_T_ate_ano_t +=1

    taxa_de_conclusao_acumulada_do_ano_T_te_ano_t = ( somatoria_de_alunos_formados_do_ano_T_ate_ano_t 
    / ( somatoria_IG_do_ano_T - somatoria_de_Fal_do_ano_T_ate_ano_t ) ) * 100

    dados_calculados_do_ano_t = {
    't' : t,
    'taxa_de_conclusao_acumulada_do_ano_T_te_ano_t' : taxa_de_conclusao_acumulada_do_ano_T_te_ano_t,
    'somatoria_de_alunos_formados_do_ano_T_ate_ano_t' : somatoria_de_alunos_formados_do_ano_T_ate_ano_t,
    'somatoria_de_Fal_do_ano_T_ate_ano_t' : somatoria_de_Fal_do_ano_T_ate_ano_t,
    'somatoria_IG_do_ano_T' : somatoria_IG_do_ano_T
    }
    return dados_calculados_do_ano_t

  #-------------------------Conteúdo da Função Principal ------------------------------
  tS = anos_de_referencia
  
  if(data_da_geracao_da_tabela.day < 31 or data_da_geracao_da_tabela.month < 12) and max(tS) >= data_da_geracao_da_tabela.year:
    return {'estado':False, 'resposta':'Erro, no caso de uma data diferente de 31/12/ano, todo ano de referência deve ser menor que o ano da data de geração da tabela.'}
  
  elif (data_da_geracao_da_tabela.day == 31 and data_da_geracao_da_tabela.month == 12) and max(tS) > data_da_geracao_da_tabela.year:
    return {'estado':False, 'resposta':'Erro, no caso de uma data igual a 31/12/ano, todo ano de referência deve ser menor ou igual ao ano da data de geração da tabela.'}

  dados_calculados_de_anos = []

  taxas_de_conclusao_acumulada_de_anos = []

  for t in tS:

    dados_calculados_do_ano_t = calcular_taxa_de_conclusao_acumulada_de_um_ano_para_um_CURSO(t, alunos_ingressados_em_T)

    dados_calculados_de_anos.append(dados_calculados_do_ano_t)

    taxas_de_conclusao_acumulada_de_anos.append(dados_calculados_do_ano_t['taxa_de_conclusao_acumulada_do_ano_T_te_ano_t'])

  dados_calculados_de_anos_por_categoria = { #Essa variável é utilizada somente numa função externa para comparar aos dados da tabela já feita
    't' : [ ] ,
    'Taxa de Conclusão Acumulada do Ano T até Ano t (%)' : [] ,
    'Somatória de Alunos Formados do Ano T ate Ano t' : [] ,
    'Somatória de Alunos Falecidos do Ano T até Ano t' : [],
    'Somatória de Alunos Ingressados em T': []
  }

  for dados_calculados_do_ano_t in dados_calculados_de_anos:

    dados_calculados_de_anos_por_categoria ['t'].append(dados_calculados_do_ano_t['t'])
    dados_calculados_de_anos_por_categoria ['Taxa de Conclusão Acumulada do Ano T até Ano t (%)'].append(dados_calculados_do_ano_t['taxa_de_conclusao_acumulada_do_ano_T_te_ano_t'])
    dados_calculados_de_anos_por_categoria ['Somatória de Alunos Formados do Ano T ate Ano t'].append(dados_calculados_do_ano_t['somatoria_de_alunos_formados_do_ano_T_ate_ano_t'])      
    dados_calculados_de_anos_por_categoria ['Somatória de Alunos Falecidos do Ano T até Ano t'].append(dados_calculados_do_ano_t['somatoria_de_Fal_do_ano_T_ate_ano_t']) 
    dados_calculados_de_anos_por_categoria ['Somatória de Alunos Ingressados em T'].append(dados_calculados_do_ano_t['somatoria_IG_do_ano_T']) 
  
  gerar_tabela_para_validar_dados(dados_calculados_de_anos_por_categoria, 'Taxa de Conclusão Acumulada dos Alunos', False)

  return {'estado':True, 'resposta': taxas_de_conclusao_acumulada_de_anos} 

###calcular_e_plotar_grafico_de_taxas_de_conclusao_acumulada_de_varios_anos_para_um_CURSO(ano_de_ingresso_dos_alunos, nome_do_curso, anos_de_referencia, data_da_geracao_da_tabela)
---
O objetivo da função é criar um gráfico para um CURSO contendo a taxa de conclusão acumulada para cada ano ao longo dos anos. Para isso utiliza a função definida no código anterior para realizar os cálculos.

In [ ]:
def calcular_e_plotar_grafico_de_taxas_de_conclusao_acumulada_de_varios_anos_para_um_CURSO(ano_de_ingresso_dos_alunos, nome_do_curso, anos_de_referencia, data_da_geracao_da_tabela, data_para_correcao_da_data_de_conclusao):
  alunos_ingressados_em_T = tratar_e_retornar_dados_de_alunos_ingressados_em_T_para_um_CURSO(ano_de_ingresso_dos_alunos, nome_do_curso, data_para_correcao_da_data_de_conclusao)
  
  resposta_da_funcao = calcular_taxas_de_conclusao_acumulada_de_varios_anos_para_um_CURSO(anos_de_referencia, alunos_ingressados_em_T, data_da_geracao_da_tabela)
  
  if resposta_da_funcao['estado'] == False:
    print(resposta_da_funcao['resposta'])
    return

  taxas_de_conclusao_acumulada_de_anos = resposta_da_funcao['resposta']

  figure = plt.figure(figsize = (9,6))
  axe = figure.add_axes([0,0,1,1])
  axe.plot(anos_de_referencia, taxas_de_conclusao_acumulada_de_anos, 's-C1')
  axe.set_xlabel('Anos')
  axe.set_ylabel('Taxas de Conclusão Acumulada de Anos')
  axe.set_title('Gráfico de Taxas de Conclusão Acumulada de Anos do Curso de '+nome_do_curso)

  incremento_x = (axe.get_xlim()[1] - axe.get_xlim()[0]) * 0.01 # para facilitar visualização dos valores plotados em relação ao eixo x, proporcional ao tamanho do eixo X
  incremento_y = (axe.get_ylim()[1] - axe.get_ylim()[0]) * 0.01 # para facilitar visualização dos valores plotados em relação ao eixo y, proporcional ao tamanho do eixo Y

  # Utilizando função text do módulo pyplot para inserir textos em coordenadas no gráfico.
  for coordenada_x, coordenada_y in zip(anos_de_referencia, taxas_de_conclusao_acumulada_de_anos):
    plt.text(coordenada_x+incremento_x, coordenada_y+incremento_y, '{:.2f}%'.format(coordenada_y))

  plt.show()

In [ ]:
#Testes
#print(nomes_dos_cursos_considerados_da_EESC)
for curso in nomes_dos_cursos_considerados_da_EESC:
  calcular_e_plotar_grafico_de_taxas_de_conclusao_acumulada_de_varios_anos_para_um_CURSO(2015, curso, [2015,2016,2017,2018,2019,2020], 
                                                                                 data_da_geracao_da_tabela = date(2021,4,21), 
                                                                                 data_para_correcao_da_data_de_conclusao = date(9999,5,31))

###calcular_taxas_de_desistencia_acumulada_de_varios_anos_para_um_CURSO(anos_de_referencia, alunos_ingressados_em_T, data_da_geracao_da_tabela)
---
O objetivo da função é calcular a taxa de desistência acumulada de cada ano ao longo dos anos para um CURSO.

In [ ]:
def calcular_taxas_de_desistencia_acumulada_de_varios_anos_para_um_CURSO(anos_de_referencia, alunos_ingressados_em_T, data_da_geracao_da_tabela):

  def calcular_taxa_de_desistencia_acumulada_de_um_ano_para_um_CURSO(t, alunos_ingressados_em_T):
    
    somatoria_IG_do_ano_T = len(alunos_ingressados_em_T) 
    
    somatoria_de_alunos_tranferidos_do_ano_T_ate_ano_t = 0

    somatoria_de_alunos_desistentes_do_ano_T_ate_ano_t = 0

    somatoria_de_Fal_do_ano_T_ate_t = 0

    for ano_de_ingresso, data_de_encerramento, tipo_de_encerramento in alunos_ingressados_em_T:

      if data_de_encerramento != '':

        if t >= data_de_encerramento.year:

          if tipo_de_encerramento.find('Transferência')!= -1:
            somatoria_de_alunos_tranferidos_do_ano_T_ate_ano_t +=1

          elif (tipo_de_encerramento.find('Abandono 2 semestres sem matrícula')!= -1 or
                tipo_de_encerramento.find('Cancelamento 0 crédito')!= -1 or
                tipo_de_encerramento.find('Cancelamento 0 crédito em dois semestres')!= -1 or
                tipo_de_encerramento.find('Cancelamento menos 20% créd 2 semestres')!= -1 or
                tipo_de_encerramento.find('Cancelamento menos 20% dos créditos')!= -1 or
                tipo_de_encerramento.find('Cancelamento trancamento 4 semestres')!= -1 or
                tipo_de_encerramento.find('Desistência a pedido')!= -1 or
                tipo_de_encerramento.find('Ingressante sem Frequência')!= -1 or
                tipo_de_encerramento.find('Encerramento novo ingresso')!= -1 or
                tipo_de_encerramento.find('Cancelamento outras IES (Lei Federal)')!= -1 or
                tipo_de_encerramento.find('Não cumpriu normas Decreto (PEC-G)')!= -1 or
                tipo_de_encerramento.find('Opção Currículo')!= -1 or
                tipo_de_encerramento.find('Liminar cassada')!= -1):
            
            somatoria_de_alunos_desistentes_do_ano_T_ate_ano_t +=1
          
          elif tipo_de_encerramento.find('Falecimento') != -1:
            somatoria_de_Fal_do_ano_T_ate_t +=1
          
    taxa_de_desistencia_acumulada_do_ano_T_ate_ano_t = ( ( somatoria_de_alunos_desistentes_do_ano_T_ate_ano_t + somatoria_de_alunos_tranferidos_do_ano_T_ate_ano_t )
    / ( somatoria_IG_do_ano_T - somatoria_de_Fal_do_ano_T_ate_t  ) ) * 100
          
    dados_calculados_do_ano_t = {
      't' : t,
      'taxa_de_desistencia_acumulada_do_ano_T_ate_ano_t' : taxa_de_desistencia_acumulada_do_ano_T_ate_ano_t ,
      'somatoria_de_alunos_tranferidos_do_ano_T_ate_ano_t' : somatoria_de_alunos_tranferidos_do_ano_T_ate_ano_t ,
      'somatoria_de_alunos_desistentes_do_ano_T_ate_ano_t' : somatoria_de_alunos_desistentes_do_ano_T_ate_ano_t,
      'somatoria_de_Fal_do_ano_T_ate_t' : somatoria_de_Fal_do_ano_T_ate_t,
      'somatoria_IG_do_ano_T' : somatoria_IG_do_ano_T,
    }

    return dados_calculados_do_ano_t

  #-------------------------Conteúdo da Função Principal ------------------------------
  
  tS = anos_de_referencia

  if(data_da_geracao_da_tabela.day < 31 or data_da_geracao_da_tabela.month < 12) and max(tS) >= data_da_geracao_da_tabela.year:
    return {'estado':False, 'resposta':'Erro, no caso de uma data diferente de 31/12/ano, todo ano de referência deve ser menor que o ano da data de geração da tabela.'}
  
  elif (data_da_geracao_da_tabela.day == 31 and data_da_geracao_da_tabela.month == 12) and max(tS) > data_da_geracao_da_tabela.year:
    return {'estado':False, 'resposta':'Erro, no caso de uma data igual a 31/12/ano, todo ano de referência deve ser menor ou igual ao ano da data de geração da tabela.'}

  dados_calculados_de_anos = []

  taxas_de_desistencia_acumulada_de_anos = []

  for t in tS:

    dados_calculados_do_ano_t = calcular_taxa_de_desistencia_acumulada_de_um_ano_para_um_CURSO(t, alunos_ingressados_em_T)

    dados_calculados_de_anos.append(dados_calculados_do_ano_t)

    taxas_de_desistencia_acumulada_de_anos.append(dados_calculados_do_ano_t['taxa_de_desistencia_acumulada_do_ano_T_ate_ano_t'])

  dados_calculados_de_anos_por_categoria = {
    't' : [ ],
    'Taxa de Desistência Acumulada do Ano T até Ano t (%)' : [] ,
    'Somatória de Alunos Transferidos do Ano T até Ano t' : [] ,
    'Somatória de Alunos Desistentes do Ano T até Ano t' : [] ,
    'Somatória de Alunos Falecidos do Ano T até Ano t' : [],
    'Somatória de Alunos Ingressados em T': []
  }

  for dados_calculados_do_ano_t in dados_calculados_de_anos:

    dados_calculados_de_anos_por_categoria ['t'].append(dados_calculados_do_ano_t['t'])
    dados_calculados_de_anos_por_categoria ['Taxa de Desistência Acumulada do Ano T até Ano t (%)'].append(dados_calculados_do_ano_t['taxa_de_desistencia_acumulada_do_ano_T_ate_ano_t'])
    dados_calculados_de_anos_por_categoria ['Somatória de Alunos Transferidos do Ano T até Ano t'].append(dados_calculados_do_ano_t['somatoria_de_alunos_tranferidos_do_ano_T_ate_ano_t'])
    dados_calculados_de_anos_por_categoria ['Somatória de Alunos Desistentes do Ano T até Ano t'].append(dados_calculados_do_ano_t['somatoria_de_alunos_desistentes_do_ano_T_ate_ano_t'])
    dados_calculados_de_anos_por_categoria ['Somatória de Alunos Falecidos do Ano T até Ano t'].append(dados_calculados_do_ano_t['somatoria_de_Fal_do_ano_T_ate_t'])
    dados_calculados_de_anos_por_categoria ['Somatória de Alunos Ingressados em T'].append(dados_calculados_do_ano_t['somatoria_IG_do_ano_T'])
    
  gerar_tabela_para_validar_dados(dados_calculados_de_anos_por_categoria, 'Taxa de Desistência Acumulada dos Alunos', False)

  return {'estado':True, 'resposta':taxas_de_desistencia_acumulada_de_anos}


###calcular_e_plotar_grafico_de_taxas_de_desistencia_acumulada_de_varios_anos_para_um_CURSO(ano_de_ingresso_dos_alunos, nome_do_curso, anos_de_referencia, data_da_geracao_da_tabela)
---
O objetivo da função é criar um gráfico para um CURSO contendo a taxa de desistencia acumulada para cada ano ao longo dos anos. Para isso utiliza a função definida no código anterior para realizar os cálculos.

In [ ]:
def calcular_e_plotar_grafico_de_taxas_de_desistencia_acumulada_de_varios_anos_para_um_CURSO(ano_de_ingresso_dos_alunos, nome_do_curso, anos_de_referencia, data_da_geracao_da_tabela, data_para_correcao_da_data_de_conclusao):

  alunos_ingressados_em_T = tratar_e_retornar_dados_de_alunos_ingressados_em_T_para_um_CURSO(ano_de_ingresso_dos_alunos, nome_do_curso, data_para_correcao_da_data_de_conclusao)
  
  resposta_da_funcao = calcular_taxas_de_desistencia_acumulada_de_varios_anos_para_um_CURSO(anos_de_referencia, alunos_ingressados_em_T, data_da_geracao_da_tabela)
  
  if resposta_da_funcao['estado'] == False:
    print(resposta_da_funcao['resposta'])
    return

  taxas_de_desistencia_acumulada_de_anos = resposta_da_funcao['resposta']

  figure = plt.figure(figsize = (9,6))
  axe = figure.add_axes([0,0,1,1])
  axe.plot(anos_de_referencia, taxas_de_desistencia_acumulada_de_anos, '^-r')
  axe.set_xlabel('Anos')
  axe.set_ylabel('Taxas de Desistência Acumulada de Anos')
  axe.set_title('Gráfico de Taxas de Desistência Acumulada de Anos do Curso de '+nome_do_curso)

  incremento_x = (axe.get_xlim()[1] - axe.get_xlim()[0]) * 0.01 # para facilitar visualização dos valores plotados em relação ao eixo x, proporcional ao tamanho do eixo X
  incremento_y = (axe.get_ylim()[1] - axe.get_ylim()[0]) * 0.01 # para facilitar visualização dos valores plotados em relação ao eixo y, proporcional ao tamanho do eixo Y

  # Utilizando função text do módulo pyplot para inserir textos em coordenadas no gráfico.
  for coordenada_x, coordenada_y in zip(anos_de_referencia, taxas_de_desistencia_acumulada_de_anos):
    plt.text(coordenada_x+incremento_x, coordenada_y+incremento_y, '{:.2f}%'.format(coordenada_y))

  plt.show()

In [ ]:
#print(nomes_dos_cursos_considerados_da_EESC)
for curso in nomes_dos_cursos_considerados_da_EESC:
  calcular_e_plotar_grafico_de_taxas_de_desistencia_acumulada_de_varios_anos_para_um_CURSO(2015, curso, [2015,2016,2017,2018,2019,2020],
                                                                                 data_da_geracao_da_tabela = date(2021,4,21), 
                                                                                 data_para_correcao_da_data_de_conclusao = date(9999,5,31))  

###calcular_e_plotar_grafico_de_indicadores_de_fluxo_de_varios_anos_para_um_CURSO(ano_de_ingresso_dos_alunos, nome_do_curso, anos_de_referencia, data_da_geracao_da_tabela)
---
O objetivo da função é criar um gráfico para um CURSO contendo a taxa de permanência, a taxa de conclusão acumulada e a taxa de desistência acumulada para cada ano ao longo dos anos.

In [ ]:
def calcular_e_plotar_grafico_de_indicadores_de_fluxo_de_varios_anos_para_um_CURSO(ano_de_ingresso_dos_alunos, nome_do_curso, anos_de_referencia, data_da_geracao_da_tabela, data_para_correcao_da_data_de_conclusao):
  
  alunos_ingressados_em_T = tratar_e_retornar_dados_de_alunos_ingressados_em_T_para_um_CURSO(ano_de_ingresso_dos_alunos, nome_do_curso, data_para_correcao_da_data_de_conclusao)
  
  resposta_da_calcular_TAP = calcular_taxas_de_permanencia_de_varios_anos_para_um_CURSO(anos_de_referencia, alunos_ingressados_em_T, data_da_geracao_da_tabela)
  resposta_da_calcular_TCA = calcular_taxas_de_conclusao_acumulada_de_varios_anos_para_um_CURSO(anos_de_referencia, alunos_ingressados_em_T, data_da_geracao_da_tabela)
  resposta_da_calcular_TDA = calcular_taxas_de_desistencia_acumulada_de_varios_anos_para_um_CURSO(anos_de_referencia, alunos_ingressados_em_T, data_da_geracao_da_tabela)
  
  if resposta_da_calcular_TAP['estado'] == False:
    print(resposta_da_calcular_TAP['resposta'])
    return
  if resposta_da_calcular_TCA['estado'] == False:
    print(resposta_da_calcular_TCA['resposta'])
    return
  if resposta_da_calcular_TDA['estado'] == False:
    print(resposta_da_calcular_TDA['resposta'])
    return

  taxas_de_permanencia_de_anos = resposta_da_calcular_TAP['resposta']
  
  taxas_de_conclusao_acumulada_de_anos = resposta_da_calcular_TCA['resposta']
  
  taxas_de_desistencia_acumulada_de_anos = resposta_da_calcular_TDA['resposta']

  figure = plt.figure(figsize = (9,6))
  axe = figure.add_axes([0,0,1,1])

  axe.plot(anos_de_referencia, taxas_de_permanencia_de_anos, 'D-b', label = "Taxa de Permanência de Anos")
  axe.plot(anos_de_referencia, taxas_de_conclusao_acumulada_de_anos, 's-C1', label = 'Taxa de Conclusão Acumulada de Anos')
  axe.plot(anos_de_referencia, taxas_de_desistencia_acumulada_de_anos, '^-r', label = 'Taxa de Desistência Acumulada de Anos')

  axe.set_xlabel('Anos')
  axe.set_ylabel('Indicadores de Fluxo de Anos')
  axe.set_title('Gráfico Indicadores de Fluxo de Anos do Curso de '+nome_do_curso)

  incremento_x = (axe.get_xlim()[1] - axe.get_xlim()[0]) * 0.01 # para facilitar visualização dos valores plotados em relação ao eixo x, proporcional ao tamanho do eixo X
  incremento_y = (axe.get_ylim()[1] - axe.get_ylim()[0]) * 0.01 # para facilitar visualização dos valores plotados em relação ao eixo y, proporcional ao tamanho do eixo Y

  # Utilizando função text do módulo pyplot para inserir textos em coordenadas no gráfico.
  for coordenada_x, coordenada_y in zip(anos_de_referencia, taxas_de_permanencia_de_anos):
    plt.text(coordenada_x+incremento_x, coordenada_y+incremento_y, '{:.2f}%'.format(coordenada_y))
  
  for coordenada_x, coordenada_y in zip(anos_de_referencia, taxas_de_conclusao_acumulada_de_anos):
    plt.text(coordenada_x+incremento_x, coordenada_y+incremento_y, '{:.2f}%'.format(coordenada_y))

  for coordenada_x, coordenada_y in zip(anos_de_referencia, taxas_de_desistencia_acumulada_de_anos):
    plt.text(coordenada_x+incremento_x, coordenada_y+incremento_y, '{:.2f}%'.format(coordenada_y))

  plt.legend()
  plt.show()

In [ ]:
#print(nomes_dos_cursos_considerados_da_EESC)
for curso in nomes_dos_cursos_considerados_da_EESC:
  calcular_e_plotar_grafico_de_indicadores_de_fluxo_de_varios_anos_para_um_CURSO(2015, curso, [2015,2016,2017,2018,2019,2020],
                                                                                 data_da_geracao_da_tabela = date(2021,4,21), 
                                                                                 data_para_correcao_da_data_de_conclusao = date(9999,5,31))  

###calcular_taxas_de_permanencia_de_varios_anos_para_uma_UNIDADE(anos_de_referencia, alunos_ingressados_em_T, data_da_geracao_da_tabela)
---
O objetivo da função é calcular a taxa de permanência de cada ano ao longo dos anos para uma UNIDADE.

In [ ]:
def calcular_taxas_de_permanencia_de_varios_anos_para_uma_UNIDADE(anos_de_referencia, alunos_ingressados_em_T, data_da_geracao_da_tabela):

  def calcular_taxa_de_permanencia_de_um_ano_para_uma_UNIDADE(t, alunos_ingressados_em_T, somatoria_de_alunos_com_vinculo_ativo_do_ano_t_menos_1):
    
    somatoria_IG_do_ano_T = len(alunos_ingressados_em_T) 
    somatoria_de_Cur_do_ano_t = None # Não é possivel calcular através do banco de dados fornecido
    somatoria_de_mTr_do_ano_t = None # Não é possivel calcular através do banco de dados fornecido
    somatoria_de_alunos_tranferidos_do_ano_t = 0
    somatoria_de_alunos_desistentes_do_ano_t = 0
    somatoria_de_alunos_formados_do_ano_t = 0
    somatoria_de_Fal_do_ano_t = 0
    somatoria_de_Fal_do_ano_T_ate_t = 0

    for ano_de_ingresso, data_de_encerramento, tipo_de_encerramento in alunos_ingressados_em_T:

      if data_de_encerramento != '':

        if t == data_de_encerramento.year:

          if (tipo_de_encerramento.find('Transferência USP')!= -1 or
              tipo_de_encerramento.find('Transferência Externa')!= -1):
              
              somatoria_de_alunos_tranferidos_do_ano_t +=1

          elif (tipo_de_encerramento.find('Abandono 2 semestres sem matrícula')!= -1 or
                tipo_de_encerramento.find('Cancelamento 0 crédito')!= -1 or
                tipo_de_encerramento.find('Cancelamento 0 crédito em dois semestres')!= -1 or
                tipo_de_encerramento.find('Cancelamento menos 20% créd 2 semestres')!= -1 or
                tipo_de_encerramento.find('Cancelamento menos 20% dos créditos')!= -1 or
                tipo_de_encerramento.find('Cancelamento trancamento 4 semestres')!= -1 or
                tipo_de_encerramento.find('Desistência a pedido')!= -1 or
                tipo_de_encerramento.find('Ingressante sem Frequência')!= -1 or
                tipo_de_encerramento.find('Encerramento novo ingresso')!= -1 or
                tipo_de_encerramento.find('Cancelamento outras IES (Lei Federal)')!= -1 or
                tipo_de_encerramento.find('Não cumpriu normas Decreto (PEC-G)')!= -1 or
                tipo_de_encerramento.find('Opção Currículo')!= -1 or
                tipo_de_encerramento.find('Liminar cassada')!= -1):
            
            somatoria_de_alunos_desistentes_do_ano_t +=1
          
          elif (tipo_de_encerramento.find('Conclusão')!= -1 or
                tipo_de_encerramento.find('Término')!= -1):
            
            somatoria_de_alunos_formados_do_ano_t +=1
          
          elif tipo_de_encerramento.find('Falecimento') != -1:
            somatoria_de_Fal_do_ano_t +=1
          
        if t >= data_de_encerramento.year and tipo_de_encerramento.find('Falecimento') != -1:
          somatoria_de_Fal_do_ano_T_ate_t +=1

    somatoria_de_alunos_com_vinculo_ativo_no_ano_t = (somatoria_de_alunos_com_vinculo_ativo_do_ano_t_menos_1
                                                                  - somatoria_de_alunos_tranferidos_do_ano_t
                                                                  - somatoria_de_alunos_desistentes_do_ano_t
                                                                  - somatoria_de_alunos_formados_do_ano_t
                                                                  - somatoria_de_Fal_do_ano_t)
    
    taxa_de_permanencia_do_ano_t = ( somatoria_de_alunos_com_vinculo_ativo_no_ano_t / (somatoria_IG_do_ano_T - somatoria_de_Fal_do_ano_T_ate_t)) * 100 

    dados_calculados_do_ano_t = {
      't' : t,
      'taxa_de_permanencia_do_ano_t' : taxa_de_permanencia_do_ano_t,
      'somatoria_de_alunos_com_vinculo_ativo_no_ano_t' : somatoria_de_alunos_com_vinculo_ativo_no_ano_t,
      'somatoria_de_alunos_tranferidos_do_ano_t' : somatoria_de_alunos_tranferidos_do_ano_t,
      'somatoria_de_alunos_desistentes_do_ano_t' : somatoria_de_alunos_desistentes_do_ano_t,
      'somatoria_de_alunos_formados_do_ano_t' : somatoria_de_alunos_formados_do_ano_t ,
      'somatoria_de_Fal_do_ano_t' : somatoria_de_Fal_do_ano_t,
      'somatoria_de_Fal_do_ano_T_ate_t' : somatoria_de_Fal_do_ano_T_ate_t,
      'somatoria_IG_do_ano_T' : somatoria_IG_do_ano_T,
    }

    return dados_calculados_do_ano_t


  #-------------------------Conteúdo da Função Principal ------------------------------

  tS = anos_de_referencia 

  if (data_da_geracao_da_tabela.day < 31 or data_da_geracao_da_tabela.month < 12) and max(tS) >= data_da_geracao_da_tabela.year:
    return {'estado':False, 'resposta':'Erro, no caso de uma data diferente de 31/12/ano, todo ano de referência deve ser menor que o ano da data de geração da tabela.'}
  
  elif (data_da_geracao_da_tabela.day == 31 and data_da_geracao_da_tabela.month == 12) and max(tS) > data_da_geracao_da_tabela.year:
    return {'estado':False, 'resposta':'Erro, no caso de uma data igual a 31/12/ano, todo ano de referência deve ser menor ou igual ao ano da data de geração da tabela.'}
    
  dados_calculados_de_anos = []

  taxas_de_permanencia_de_anos = []

  somatoria_de_alunos_com_vinculo_ativo_do_ano_t_menos_1 = len(alunos_ingressados_em_T) # Essa atribuição é feita pois no ano T, não existe t-1

  for t in tS:

    dados_calculados_do_ano_t = calcular_taxa_de_permanencia_de_um_ano_para_uma_UNIDADE(t, alunos_ingressados_em_T, somatoria_de_alunos_com_vinculo_ativo_do_ano_t_menos_1)

    dados_calculados_de_anos.append(dados_calculados_do_ano_t)

    taxas_de_permanencia_de_anos.append(dados_calculados_do_ano_t['taxa_de_permanencia_do_ano_t'])

    somatoria_de_alunos_com_vinculo_ativo_do_ano_t_menos_1 = dados_calculados_do_ano_t['somatoria_de_alunos_com_vinculo_ativo_no_ano_t']

    dados_calculados_de_anos_por_categoria = {
      't' : [ ],
      'Taxa de Permanência do Ano t (%)' : [] ,
      'Somatória de Alunos com Vínculo Ativo do Ano t' : [] ,
      'Somatória de Alunos Transferidos(Externa e USP) do Ano t' : [] ,
      'Somatória de Alunos Desistentes do Ano t' : [] ,
      'Somatória de Alunos Formados do Ano t' : [] ,
      'Somatória de Alunos Falecidos do Ano t' : [],
      'Somatória de Alunos Falecidos do Ano T até Ano t' : [],
      'Somatória de Alunos Ingressados em T': [] 
    }

  for dados_calculados_do_ano_t in dados_calculados_de_anos:

    dados_calculados_de_anos_por_categoria ['t'].append(dados_calculados_do_ano_t['t'])
    dados_calculados_de_anos_por_categoria ['Taxa de Permanência do Ano t (%)'].append(dados_calculados_do_ano_t['taxa_de_permanencia_do_ano_t'])
    dados_calculados_de_anos_por_categoria ['Somatória de Alunos com Vínculo Ativo do Ano t'].append(dados_calculados_do_ano_t['somatoria_de_alunos_com_vinculo_ativo_no_ano_t'])
    dados_calculados_de_anos_por_categoria ['Somatória de Alunos Transferidos(Externa e USP) do Ano t'].append(dados_calculados_do_ano_t['somatoria_de_alunos_tranferidos_do_ano_t'])
    dados_calculados_de_anos_por_categoria ['Somatória de Alunos Desistentes do Ano t'].append(dados_calculados_do_ano_t['somatoria_de_alunos_desistentes_do_ano_t'])
    dados_calculados_de_anos_por_categoria ['Somatória de Alunos Formados do Ano t'].append(dados_calculados_do_ano_t['somatoria_de_alunos_formados_do_ano_t'])
    dados_calculados_de_anos_por_categoria ['Somatória de Alunos Falecidos do Ano t'].append(dados_calculados_do_ano_t['somatoria_de_Fal_do_ano_t'])
    dados_calculados_de_anos_por_categoria ['Somatória de Alunos Falecidos do Ano T até Ano t'].append(dados_calculados_do_ano_t['somatoria_de_Fal_do_ano_T_ate_t'])
    dados_calculados_de_anos_por_categoria ['Somatória de Alunos Ingressados em T'].append(dados_calculados_do_ano_t['somatoria_IG_do_ano_T'])

  gerar_tabela_para_validar_dados(dados_calculados_de_anos_por_categoria, 'Taxa de Permanência dos Alunos', False)

  return {'estado':True, 'resposta':taxas_de_permanencia_de_anos}

###calcular_e_plotar_grafico_de_taxas_de_permanencia_de_varios_anos_para_uma_UNIDADE(ano_de_ingresso_dos_alunos, nome_do_curso, anos_de_referencia, data_da_geracao_da_tabela)
---
O objetivo da função é criar um gráfico para uma UNIDADE contendo a taxa de permanência para cada ano ao longo dos anos. Para isso utiliza a função definida no código anterior para realizar os cálculos.

In [ ]:
def calcular_e_plotar_grafico_de_taxas_de_permanencia_de_varios_anos_para_uma_UNIDADE(ano_de_ingresso_dos_alunos, codigo_da_unidade, anos_de_referencia, data_da_geracao_da_tabela, data_para_correcao_da_data_de_conclusao):
  
  alunos_ingressados_em_T = tratar_e_retornar_dados_de_alunos_ingressados_em_T_para_uma_UNIDADE(ano_de_ingresso_dos_alunos, codigo_da_unidade, data_para_correcao_da_data_de_conclusao)
  
  resposta_da_funcao = calcular_taxas_de_permanencia_de_varios_anos_para_uma_UNIDADE(anos_de_referencia, alunos_ingressados_em_T, data_da_geracao_da_tabela)
  
  if resposta_da_funcao['estado'] == False:
    print(resposta_da_funcao['resposta'])
    return

  taxas_de_permanencia_de_anos = resposta_da_funcao['resposta']

  figure = plt.figure(figsize = (9,6))
  axe = figure.add_axes([0,0,1,1])
  axe.plot(anos_de_referencia, taxas_de_permanencia_de_anos, 'D-b')
  axe.set_xlabel('Anos')
  axe.set_ylabel('Taxas de Permanência de Anos')
  axe.set_title('Gráfico de Taxas de Permanência de Anos da unidade '+codigo_da_unidade)

  incremento_x = (axe.get_xlim()[1] - axe.get_xlim()[0]) * 0.01 # para facilitar visualização dos valores plotados em relação ao eixo x, proporcional ao tamanho do eixo X
  incremento_y = (axe.get_ylim()[1] - axe.get_ylim()[0]) * 0.01 # para facilitar visualização dos valores plotados em relação ao eixo y, proporcional ao tamanho do eixo Y

  # Utilizando função text do módulo pyplot para inserir textos em coordenadas no gráfico.
  for coordenada_x, coordenada_y in zip(anos_de_referencia, taxas_de_permanencia_de_anos):
    plt.text(coordenada_x+incremento_x, coordenada_y+incremento_y, '{:.2f}%'.format(coordenada_y))

  plt.show()

In [ ]:
#Testes
#print(nomes_dos_cursos_considerados_da_EESC)
calcular_e_plotar_grafico_de_taxas_de_permanencia_de_varios_anos_para_uma_UNIDADE(2015, '18', [2015,2016,2017,2018,2019,2020],
                                                                                 data_da_geracao_da_tabela = date(2021,4,21), 
                                                                                 data_para_correcao_da_data_de_conclusao = date(9999,5,31))

###calcular_taxas_de_conclusao_acumulada_de_varios_anos_para_uma_UNIDADE(anos_de_referencia, alunos_ingressados_em_T, data_da_geracao_da_tabela)
---
O objetivo da função é calcular a taxa de conclusão acumulada de cada ano ao longo dos anos para uma UNIDADE.

In [ ]:
def calcular_taxas_de_conclusao_acumulada_de_varios_anos_para_uma_UNIDADE(anos_de_referencia, alunos_ingressados_em_T, data_da_geracao_da_tabela):

  def calcular_taxa_de_conclusao_acumulada_de_um_ano_para_uma_UNIDADE(t, alunos_ingressados_em_T):
    
    somatoria_IG_do_ano_T = len(alunos_ingressados_em_T)
    somatoria_de_alunos_formados_do_ano_T_ate_ano_t = 0
    somatoria_de_Fal_do_ano_T_ate_ano_t = 0 

    for ano_de_ingresso, data_de_encerramento, tipo_de_encerramento in alunos_ingressados_em_T:

      if data_de_encerramento != '':

        if t >= data_de_encerramento.year:

          if (tipo_de_encerramento.find('Conclusão')!= -1 or
                tipo_de_encerramento.find('Término')!= -1):
                  somatoria_de_alunos_formados_do_ano_T_ate_ano_t +=1
          
          elif tipo_de_encerramento.find('Falecimento') != -1:
            somatoria_de_Fal_do_ano_T_ate_ano_t +=1

    taxa_de_conclusao_acumulada_do_ano_T_te_ano_t = ( somatoria_de_alunos_formados_do_ano_T_ate_ano_t 
    / ( somatoria_IG_do_ano_T - somatoria_de_Fal_do_ano_T_ate_ano_t ) ) * 100

    dados_calculados_do_ano_t = {
    't' : t,
    'taxa_de_conclusao_acumulada_do_ano_T_te_ano_t' : taxa_de_conclusao_acumulada_do_ano_T_te_ano_t,
    'somatoria_de_Fal_do_ano_T_ate_ano_t' : somatoria_de_Fal_do_ano_T_ate_ano_t,
    'somatoria_IG_do_ano_T' : somatoria_IG_do_ano_T
    }
    return dados_calculados_do_ano_t

  #-------------------------Conteúdo da Função Principal ------------------------------
  tS = anos_de_referencia
  
  if(data_da_geracao_da_tabela.day < 31 or data_da_geracao_da_tabela.month < 12) and max(tS) >= data_da_geracao_da_tabela.year:
    return {'estado':False, 'resposta':'Erro, no caso de uma data diferente de 31/12/ano, todo ano de referência deve ser menor que o ano da data de geração da tabela.'}
  
  elif (data_da_geracao_da_tabela.day == 31 and data_da_geracao_da_tabela.month == 12) and max(tS) > data_da_geracao_da_tabela.year:
    return {'estado':False, 'resposta':'Erro, no caso de uma data igual a 31/12/ano, todo ano de referência deve ser menor ou igual ao ano da data de geração da tabela.'}

  dados_calculados_de_anos = []

  taxas_de_conclusao_acumulada_de_anos = []

  for t in tS:

    dados_calculados_do_ano_t = calcular_taxa_de_conclusao_acumulada_de_um_ano_para_uma_UNIDADE(t, alunos_ingressados_em_T)

    dados_calculados_de_anos.append(dados_calculados_do_ano_t)

    taxas_de_conclusao_acumulada_de_anos.append(dados_calculados_do_ano_t['taxa_de_conclusao_acumulada_do_ano_T_te_ano_t'])

  dados_calculados_de_anos_por_categoria = { #Essa variável é utilizada somente numa função externa para comparar aos dados da tabela já feita
    't' : [] ,
    'Taxa de Conclusão Acumulada do Ano T até Ano t (%)' : [] ,
    'Somatória de Alunos Falecidos do Ano T até Ano t' : [], 
    'Somatória de Alunos Ingressados em T': []
  }

  for dados_calculados_do_ano_t in dados_calculados_de_anos:

    dados_calculados_de_anos_por_categoria ['t'].append(dados_calculados_do_ano_t['t'])
    dados_calculados_de_anos_por_categoria ['Taxa de Conclusão Acumulada do Ano T até Ano t (%)'].append(dados_calculados_do_ano_t['taxa_de_conclusao_acumulada_do_ano_T_te_ano_t'])      
    dados_calculados_de_anos_por_categoria ['Somatória de Alunos Falecidos do Ano T até Ano t'].append(dados_calculados_do_ano_t['somatoria_de_Fal_do_ano_T_ate_ano_t'])
    dados_calculados_de_anos_por_categoria ['Somatória de Alunos Ingressados em T'].append(dados_calculados_do_ano_t['somatoria_IG_do_ano_T'])  
  
  gerar_tabela_para_validar_dados(dados_calculados_de_anos_por_categoria, 'Taxa de Conclusão Acumulada dos Alunos', False)

  return {'estado':True, 'resposta': taxas_de_conclusao_acumulada_de_anos}

###calcular_e_plotar_grafico_de_taxas_de_conclusao_acumulada_de_varios_anos_para_uma_UNIDADE(ano_de_ingresso_dos_alunos, nome_do_curso, anos_de_referencia, data_da_geracao_da_tabela)
---
O objetivo da função é criar um gráfico para uma UNIDADE contendo a taxa de conclusão acumulada para cada ano ao longo dos anos. Para isso utiliza a função definida no código anterior para realizar os cálculos.

In [ ]:
def calcular_e_plotar_grafico_de_taxas_de_conclusao_acumulada_de_varios_anos_para_uma_UNIDADE(ano_de_ingresso_dos_alunos, codigo_da_unidade, anos_de_referencia, data_da_geracao_da_tabela, data_para_correcao_da_data_de_conclusao):
  
  alunos_ingressados_em_T = tratar_e_retornar_dados_de_alunos_ingressados_em_T_para_uma_UNIDADE(ano_de_ingresso_dos_alunos, codigo_da_unidade, data_para_correcao_da_data_de_conclusao)
  
  resposta_da_funcao = calcular_taxas_de_conclusao_acumulada_de_varios_anos_para_uma_UNIDADE(anos_de_referencia, alunos_ingressados_em_T, data_da_geracao_da_tabela)
  
  if resposta_da_funcao['estado'] == False:
    print(resposta_da_funcao['resposta'])
    return

  taxas_de_conclusao_acumulada_de_anos = resposta_da_funcao['resposta']

  figure = plt.figure(figsize = (9,6))
  axe = figure.add_axes([0,0,1,1])
  axe.plot(anos_de_referencia, taxas_de_conclusao_acumulada_de_anos, 's-C1')
  axe.set_xlabel('Anos')
  axe.set_ylabel('Taxas de Conclusão Acumulada de Anos')
  axe.set_title('Gráfico de Taxas de Conclusão Acumulada de Anos da Unidade '+codigo_da_unidade)

  incremento_x = (axe.get_xlim()[1] - axe.get_xlim()[0]) * 0.01 # para facilitar visualização dos valores plotados em relação ao eixo x, proporcional ao tamanho do eixo X
  incremento_y = (axe.get_ylim()[1] - axe.get_ylim()[0]) * 0.01 # para facilitar visualização dos valores plotados em relação ao eixo y, proporcional ao tamanho do eixo Y

  # Utilizando função text do módulo pyplot para inserir textos em coordenadas no gráfico.
  for coordenada_x, coordenada_y in zip(anos_de_referencia, taxas_de_conclusao_acumulada_de_anos):
    plt.text(coordenada_x+incremento_x, coordenada_y+incremento_y, '{:.2f}%'.format(coordenada_y))

  plt.show()

In [ ]:
#Testes
#print(nomes_dos_cursos_considerados_da_EESC)
calcular_e_plotar_grafico_de_taxas_de_conclusao_acumulada_de_varios_anos_para_uma_UNIDADE(2015, '18', [2015,2016,2017,2018,2019,2020],
                                                                                 data_da_geracao_da_tabela = date(2021,4,21), 
                                                                                 data_para_correcao_da_data_de_conclusao = date(9999,5,31))

###calcular_taxas_de_desistencia_acumulada_de_varios_anos_para_uma_UNIDADE(anos_de_referencia, alunos_ingressados_em_T, data_da_geracao_da_tabela)
---
O objetivo da função é calcular a taxa de desistência acumulada de cada ano ao longo dos anos para uma UNIDADE.

In [ ]:
def calcular_taxas_de_desistencia_acumulada_de_varios_anos_para_uma_UNIDADE(anos_de_referencia, alunos_ingressados_em_T, data_da_geracao_da_tabela):

  def calcular_taxa_de_desistencia_acumulada_de_um_ano_para_uma_UNIDADE(t, alunos_ingressados_em_T):
    
    somatoria_IG_do_ano_T = len(alunos_ingressados_em_T) 
    
    somatoria_de_alunos_tranferidos_do_ano_T_ate_ano_t = 0

    somatoria_de_alunos_desistentes_do_ano_T_ate_ano_t = 0

    somatoria_de_Fal_do_ano_T_ate_t = 0

    for ano_de_ingresso, data_de_encerramento, tipo_de_encerramento in alunos_ingressados_em_T:

      if data_de_encerramento != '':

        if t >= data_de_encerramento.year:

          if (tipo_de_encerramento.find('Transferência USP')!= -1 or
              tipo_de_encerramento.find('Transferência Externa')!= -1):
            
            somatoria_de_alunos_tranferidos_do_ano_T_ate_ano_t +=1

          elif (tipo_de_encerramento.find('Abandono 2 semestres sem matrícula')!= -1 or
                tipo_de_encerramento.find('Cancelamento 0 crédito')!= -1 or
                tipo_de_encerramento.find('Cancelamento 0 crédito em dois semestres')!= -1 or
                tipo_de_encerramento.find('Cancelamento menos 20% créd 2 semestres')!= -1 or
                tipo_de_encerramento.find('Cancelamento menos 20% dos créditos')!= -1 or
                tipo_de_encerramento.find('Cancelamento trancamento 4 semestres')!= -1 or
                tipo_de_encerramento.find('Desistência a pedido')!= -1 or
                tipo_de_encerramento.find('Ingressante sem Frequência')!= -1 or
                tipo_de_encerramento.find('Encerramento novo ingresso')!= -1 or
                tipo_de_encerramento.find('Cancelamento outras IES (Lei Federal)')!= -1 or
                tipo_de_encerramento.find('Não cumpriu normas Decreto (PEC-G)')!= -1 or
                tipo_de_encerramento.find('Opção Currículo')!= -1 or
                tipo_de_encerramento.find('Liminar cassada')!= -1):
            
            somatoria_de_alunos_desistentes_do_ano_T_ate_ano_t +=1
          
          elif tipo_de_encerramento.find('Falecimento') != -1:
            somatoria_de_Fal_do_ano_T_ate_t +=1
          
    taxa_de_desistencia_acumulada_do_ano_T_ate_ano_t = ( ( somatoria_de_alunos_desistentes_do_ano_T_ate_ano_t + somatoria_de_alunos_tranferidos_do_ano_T_ate_ano_t )
    / ( somatoria_IG_do_ano_T - somatoria_de_Fal_do_ano_T_ate_t  ) ) * 100
          
    dados_calculados_do_ano_t = {
      't' : t,
      'taxa_de_desistencia_acumulada_do_ano_T_ate_ano_t' : taxa_de_desistencia_acumulada_do_ano_T_ate_ano_t ,
      'somatoria_de_alunos_tranferidos_do_ano_T_ate_ano_t' : somatoria_de_alunos_tranferidos_do_ano_T_ate_ano_t ,
      'somatoria_de_alunos_desistentes_do_ano_T_ate_ano_t' : somatoria_de_alunos_desistentes_do_ano_T_ate_ano_t,
      'somatoria_de_Fal_do_ano_T_ate_t' : somatoria_de_Fal_do_ano_T_ate_t,
      'somatoria_IG_do_ano_T' : somatoria_IG_do_ano_T,
    }

    return dados_calculados_do_ano_t

  #-------------------------Conteúdo da Função Principal ------------------------------
  
  tS = anos_de_referencia

  if(data_da_geracao_da_tabela.day < 31 or data_da_geracao_da_tabela.month < 12) and max(tS) >= data_da_geracao_da_tabela.year:
    return {'estado':False, 'resposta':'Erro, no caso de uma data diferente de 31/12/ano, todo ano de referência deve ser menor que o ano da data de geração da tabela.'}
  
  elif (data_da_geracao_da_tabela.day == 31 and data_da_geracao_da_tabela.month == 12) and max(tS) > data_da_geracao_da_tabela.year:
    return {'estado':False, 'resposta':'Erro, no caso de uma data igual a 31/12/ano, todo ano de referência deve ser menor ou igual ao ano da data de geração da tabela.'}

  dados_calculados_de_anos = []

  taxas_de_desistencia_acumulada_de_anos = []

  for t in tS:

    dados_calculados_do_ano_t = calcular_taxa_de_desistencia_acumulada_de_um_ano_para_uma_UNIDADE(t, alunos_ingressados_em_T)

    dados_calculados_de_anos.append(dados_calculados_do_ano_t)

    taxas_de_desistencia_acumulada_de_anos.append(dados_calculados_do_ano_t['taxa_de_desistencia_acumulada_do_ano_T_ate_ano_t'])

  dados_calculados_de_anos_por_categoria = {
    't' : [ ],
    'Taxa de Desistência Acumulada do Ano T até Ano t (%)' : [] ,
    'Somatória de Alunos Transferidos do Ano T até Ano t' : [] ,
    'Somatória de Alunos Desistentes do Ano T até Ano t' : [] ,
    'Somatória de Alunos Falecidos do Ano T até Ano t' : [],
    'Somatória de Alunos Ingressados em T': []
  }

  for dados_calculados_do_ano_t in dados_calculados_de_anos:

    dados_calculados_de_anos_por_categoria ['t'].append(dados_calculados_do_ano_t['t'])
    dados_calculados_de_anos_por_categoria ['Taxa de Desistência Acumulada do Ano T até Ano t (%)'].append(dados_calculados_do_ano_t['taxa_de_desistencia_acumulada_do_ano_T_ate_ano_t'])
    dados_calculados_de_anos_por_categoria ['Somatória de Alunos Transferidos do Ano T até Ano t'].append(dados_calculados_do_ano_t['somatoria_de_alunos_tranferidos_do_ano_T_ate_ano_t'])
    dados_calculados_de_anos_por_categoria ['Somatória de Alunos Desistentes do Ano T até Ano t'].append(dados_calculados_do_ano_t['somatoria_de_alunos_desistentes_do_ano_T_ate_ano_t'])
    dados_calculados_de_anos_por_categoria ['Somatória de Alunos Falecidos do Ano T até Ano t'].append(dados_calculados_do_ano_t['somatoria_de_Fal_do_ano_T_ate_t'])
    dados_calculados_de_anos_por_categoria ['Somatória de Alunos Ingressados em T'].append(dados_calculados_do_ano_t['somatoria_IG_do_ano_T'])
    
  gerar_tabela_para_validar_dados(dados_calculados_de_anos_por_categoria, 'Taxa de Desistência Acumulada dos Alunos', False)

  return {'estado':True, 'resposta':taxas_de_desistencia_acumulada_de_anos}

###calcular_e_plotar_grafico_de_taxas_de_desistencia_acumulada_de_varios_anos_para_uma_UNIDADE(ano_de_ingresso_dos_alunos, nome_do_curso, anos_de_referencia, data_da_geracao_da_tabela)
---
O objetivo da função é criar um gráfico para uma UNIDADE contendo a taxa de desistencia acumulada para cada ano ao longo dos anos. Para isso utiliza a função definida no código anterior para realizar os cálculos.

In [ ]:
def calcular_e_plotar_grafico_de_taxas_de_desistencia_acumulada_de_varios_anos_para_uma_UNIDADE(ano_de_ingresso_dos_alunos, codigo_da_unidade, anos_de_referencia, data_da_geracao_da_tabela, data_para_correcao_da_data_de_conclusao):

  alunos_ingressados_em_T = tratar_e_retornar_dados_de_alunos_ingressados_em_T_para_uma_UNIDADE(ano_de_ingresso_dos_alunos, codigo_da_unidade, data_para_correcao_da_data_de_conclusao)
  
  resposta_da_funcao = calcular_taxas_de_desistencia_acumulada_de_varios_anos_para_uma_UNIDADE(anos_de_referencia, alunos_ingressados_em_T, data_da_geracao_da_tabela)
  
  if resposta_da_funcao['estado'] == False:
    print(resposta_da_funcao['resposta'])
    return

  taxas_de_desistencia_acumulada_de_anos = resposta_da_funcao['resposta']

  figure = plt.figure(figsize = (9,6))
  axe = figure.add_axes([0,0,1,1])
  axe.plot(anos_de_referencia, taxas_de_desistencia_acumulada_de_anos, '^-r')
  axe.set_xlabel('Anos')
  axe.set_ylabel('Taxas de Desistência Acumulada de Anos')
  axe.set_title('Gráfico de Taxas de Desistência Acumulada de Anos da Unidade '+codigo_da_unidade)

  incremento_x = (axe.get_xlim()[1] - axe.get_xlim()[0]) * 0.01 # para facilitar visualização dos valores plotados em relação ao eixo x, proporcional ao tamanho do eixo X
  incremento_y = (axe.get_ylim()[1] - axe.get_ylim()[0]) * 0.01 # para facilitar visualização dos valores plotados em relação ao eixo y, proporcional ao tamanho do eixo Y

  # Utilizando função text do módulo pyplot para inserir textos em coordenadas no gráfico.
  for coordenada_x, coordenada_y in zip(anos_de_referencia, taxas_de_desistencia_acumulada_de_anos):
    plt.text(coordenada_x+incremento_x, coordenada_y+incremento_y, '{:.2f}%'.format(coordenada_y))

  plt.show()

In [ ]:
#Testes
#print(nomes_dos_cursos_considerados_da_EESC)
calcular_e_plotar_grafico_de_taxas_de_desistencia_acumulada_de_varios_anos_para_uma_UNIDADE(2015, '18', [2015,2016,2017,2018,2019,2020],
                                                                                    data_da_geracao_da_tabela = date(2021,4,21),
                                                                                    data_para_correcao_da_data_de_conclusao = date(9999,5,31))

###calcular_e_plotar_grafico_de_indicadores_de_fluxo_de_varios_anos_para_uma_UNIDADE(ano_de_ingresso_dos_alunos, nome_do_curso, anos_de_referencia, data_da_geracao_da_tabela)
---
O objetivo da função é criar um gráfico para uma UNIDADE contendo a taxa de permanência, a taxa de conclusão acumulada e a taxa de desistência acumulada para cada ano ao longo dos anos.

In [ ]:
def calcular_e_plotar_grafico_de_indicadores_de_fluxo_de_varios_anos_para_uma_UNIDADE(ano_de_ingresso_dos_alunos, codigo_da_unidade, anos_de_referencia, data_da_geracao_da_tabela, data_para_correcao_da_data_de_conclusao):
  
  alunos_ingressados_em_T = tratar_e_retornar_dados_de_alunos_ingressados_em_T_para_uma_UNIDADE(ano_de_ingresso_dos_alunos, codigo_da_unidade, data_para_correcao_da_data_de_conclusao)
  
  resposta_da_funcao1 = calcular_taxas_de_permanencia_de_varios_anos_para_uma_UNIDADE(anos_de_referencia, alunos_ingressados_em_T, data_da_geracao_da_tabela)
  resposta_da_funcao2 = calcular_taxas_de_conclusao_acumulada_de_varios_anos_para_uma_UNIDADE(anos_de_referencia, alunos_ingressados_em_T, data_da_geracao_da_tabela)
  resposta_da_funcao3 = calcular_taxas_de_desistencia_acumulada_de_varios_anos_para_uma_UNIDADE(anos_de_referencia, alunos_ingressados_em_T, data_da_geracao_da_tabela)
  
  if resposta_da_funcao1['estado'] == False:
    print(resposta_da_funcao1['resposta'])
    return
  if resposta_da_funcao2['estado'] == False:
    print(resposta_da_funcao2['resposta'])
    return
  if resposta_da_funcao3['estado'] == False:
    print(resposta_da_funcao3['resposta'])
    return

  taxas_de_permanencia_de_anos = resposta_da_funcao1['resposta']
  
  taxas_de_conclusao_acumulada_de_anos = resposta_da_funcao2['resposta']
  
  taxas_de_desistencia_acumulada_de_anos = resposta_da_funcao3['resposta']

  figure = plt.figure(figsize = (9,6))
  axe = figure.add_axes([0,0,1,1])

  axe.plot(anos_de_referencia, taxas_de_permanencia_de_anos, 'D-b', label = "Taxa de Permanência de Anos")
  axe.plot(anos_de_referencia, taxas_de_conclusao_acumulada_de_anos, 's-C1', label = 'Taxa de Conclusão Acumulada de Anos')
  axe.plot(anos_de_referencia, taxas_de_desistencia_acumulada_de_anos, '^-r', label = 'Taxa de Desistência Acumulada de Anos')

  axe.set_xlabel('Anos')
  axe.set_ylabel('Indicadores de Fluxo de Anos')
  axe.set_title('Gráfico Indicadores de Fluxo de Anos da Unidade '+codigo_da_unidade)

  incremento_x = (axe.get_xlim()[1] - axe.get_xlim()[0]) * 0.01 # para facilitar visualização dos valores plotados em relação ao eixo x, proporcional ao tamanho do eixo X
  incremento_y = (axe.get_ylim()[1] - axe.get_ylim()[0]) * 0.01 # para facilitar visualização dos valores plotados em relação ao eixo y, proporcional ao tamanho do eixo Y

  # Utilizando função text do módulo pyplot para inserir textos em coordenadas no gráfico.
  for coordenada_x, coordenada_y in zip(anos_de_referencia, taxas_de_permanencia_de_anos):
    plt.text(coordenada_x+incremento_x, coordenada_y+incremento_y, '{:.2f}%'.format(coordenada_y))
  
  for coordenada_x, coordenada_y in zip(anos_de_referencia, taxas_de_conclusao_acumulada_de_anos):
    plt.text(coordenada_x+incremento_x, coordenada_y+incremento_y, '{:.2f}%'.format(coordenada_y))

  for coordenada_x, coordenada_y in zip(anos_de_referencia, taxas_de_desistencia_acumulada_de_anos):
    plt.text(coordenada_x+incremento_x, coordenada_y+incremento_y, '{:.2f}%'.format(coordenada_y))

  plt.legend()
  plt.show()

In [ ]:
#Testes
calcular_e_plotar_grafico_de_indicadores_de_fluxo_de_varios_anos_para_uma_UNIDADE(2015, '18', [2015,2016,2017,2018,2019,2020],
                                                                                 data_da_geracao_da_tabela = date(2021,4,21), 
                                                                                 data_para_correcao_da_data_de_conclusao = date(9999,5,31))